## α-β-Pruning mit Rote-Learning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-artificial-intelligence.ipynb
%run ./nmm-heuristic.ipynb
%run ./nmm-symmetry.ipynb
%run ./nmm-cache-rote-learning.ipynb
import time

Für die Implementierung des *α-β-Pruning* Algorithmus mit *Rote-Learning* wird eine Klasse `AlphaBetaPruningRoteLearning` implementiert, die ebenfalls von der zuvor definierten Klasse `ArtificialIntelligence` erbt. Auch hier wird die Funktion `bestMoves` überschrieben, die später genauer definiert wird, und den Konstruktor `__init__` implementiert. Der Konstruktor besitzt drei optionale Parameter, die den Algorithmus konfigurieren können:

* die `limit` Einstellung setzt die maximale Rekursionstiefe;
* die `symmetry` Einstellung legt fest, ob alle symmetrischen Spielfelder zu einem Zustand berechnet werden sollen und dann ebenfalls in der Transpositionstabelle abgelegt werden sollen;
* durch den `weights` Parameter können die Gewichtungen der zuvor definierten Heuristik bestimmt werden;
* durch den `cache` Parameter kann ein Cache übergeben, falls dies nicht der Fall ist wird ein neuer leerer Cache initialisert.

In [ ]:
class AlphaBetaPruningRoteLearning(ArtificialIntelligence):
    def __init__(self, limit=3, symmetry=True, weights=None, cache=None, name=None):
        
        self.name = name
        self.cache = cache
        if self.cache is None:
            self.cache = CacheRoteLearning()
        
        self.limit = limit
        self.symmetry = symmetry
        self.weights = weights
        if self.weights is None:
            self.weights = HeuristicWeights()
    
    def bestMoves(self, state, player):
        pass

Für Debuggingzwecke wird eine `__repr__` Funktion implementiert, die eine String-Repräsentation aus einer `AlphaBetaPruning` Instanz mit allen Einstelungen erstellt.

In [ ]:
def __repr__(self: AlphaBetaPruningRoteLearning):
    return f"AlphaBetaPruningRoteLearning(name={self.name}, limit={self.limit}, symmetry={self.symmetry}, weights={self.weights})"

AlphaBetaPruningRoteLearning.__repr__ = __repr__
del __repr__

### Implementierung *α-β-Pruning*
#### Hilfsfunktion zum Füllen der Transpositionstabelle `writeCache`

Die Hilfsfunktion `writeCache` legt den gegebenen Zustand `state` und Spieler `player` mit samt des errechneten Wertes `value` in der Transpositionstabelle (`cache`) ab. Ist zusätzlich die Einstellung `symmetry` aktiviert, werden auch die errechneten Spielfelder in der Transpositionstabelle gespeichert.

In [ ]:
def writeCache(self, state, player, value):
    if self.symmetry:
        for symmetricState in findSymmetries(state):
            self.cache.write(symmetricState, player, value)
    else:
        self.cache.write(state, player, value)

AlphaBetaPruningRoteLearning.writeCache = writeCache
del writeCache

#### Funktion zur Berechnung des Wertes eines Spielzustands `alphaBeta`

Die Funktion `alphaBeta` beinhaltet die Implementierung des *α-β-Pruning* verbunden mit dem Rote-Learning.

* Wie zuvor beim *Minimax-Algorithmus*, wird der `utility` Wert zurückgegeben, falls das Spiel in dem State `s` beendet (`finished`) ist.
* Ebenfalls äquivalent wird der `heuristic` Wert verwendet, sobald das Rekursionslimit (`limit`) erreicht wird. Allerdings kann hier der Wert aus dem Cache geladen werden, falls dieser zur Verfügung steht.
* Der eigentliche α-β-Pruning Alogrithmus errechnet rekursiv den Wert eines Zuges. Hierbei wird der erste Wert der nächsten States verwendet, der größer oder gleich der oberen Grenze `beta` ist.

In [ ]:
def alphaBeta(self, state, player, alpha, beta, limit):
    if limit == 0:
        value = self.cache.read(state, player)
        if value:
            self.cache_hit += 1
            return value
        else:
            self.cache_miss += 1
            return heuristic(state, player, self.weights)

    states = nextStates(state, player)
    if finished(state, player, ns=states):
        return utility(state, player, ns=states)

    val = alpha
    for ns in states:
        val = max(val, -self.alphaBeta(ns, opponent(player), -beta, -alpha, limit-1))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

AlphaBetaPruningRoteLearning.alphaBeta = alphaBeta
del alphaBeta

#### Funktion zur Auswahl des bestmöglichen Zuges `bestMoves`

Die Funktion `bestMoves` berechnet rekursiv den geschätzten Wert aller möglichen Züge für einen Spieler `player` und wählt die Züge mit dem besten Wert aus. Diese rekursive Suche wird solange ausgeführt, bis die maximale Rekursionstiefe erreicht ist. Abschließend werden aus den erhaltenen Zuständen die Zustände mit dem besten Wert ausgewählt und zurückgegeben.

Zusätzlich werden Informationen gesammelt, die die genaueren Abläufe im Algorithmus abbilden:
* `runtime` ist die Rechendauer der Funktion in Sekunden;
* `limit` ist die erreichte maximale Rekursionstiefe;
* `visited` ist die Anzahl der besuchten Zustände;
* `cache_hit` ist die Anzahl der Berechenungen, die durch die Tanspositionstabelle (`cache`) eingespart werden konnten;
* `cache_miss` ist die Anzahl der Berechnungen, die trotz der Transpositionstabelle durchgeführt werden mussten, da kein Eintrag gefunden wurde;
* `cache_size` ist die Anzahl der Zustände im Cache.

In [ ]:
def bestMoves(self, state, player):
    # Start clock
    start = time.time()
    
    # Reset counter
    self.visited = 0
    self.cache_hit = 0
    self.cache_miss = 0
    
    states = nextStates(state, player)
    moves = [
        (-self.alphaBeta(s, opponent(player), -1, 1, self.limit), s)
        for s in states
    ]
    maximum = max(v for (v, s) in moves)
    
    self.writeCache(state, player, maximum)
    bestMoves = [s for (v, s) in moves if v == maximum]
    
    end = time.time()
    return BestMoves(
        bestMoves,
        maximum,
        # Collect debug information
        {   "runtime": end - start,
            "limit": self.limit,
            "visited": self.visited,
            "cache_hit": self.cache_hit,
            "cache_miss": self.cache_miss,   
            "cache_size": len(self.cache.cache)
        }
    )

AlphaBetaPruningRoteLearning.bestMoves = bestMoves
del bestMoves